In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("MSFT_from_2015_minute.csv", index_col=['timestamp']) #nrows=10000, parse_dates=['timestamp'], infer_datetime_format=True)
# five_min_closes = df['close'].resample("5T").mean()
print(df)

                             open    high     low   close   volume
timestamp                                                         
2015-01-02 04:00:00-05:00   46.50   46.50   46.50   46.50    100.0
2015-01-02 07:05:00-05:00   46.72   46.72   46.72   46.72    217.0
2015-01-02 07:29:00-05:00   46.70   46.70   46.70   46.70    100.0
2015-01-02 08:00:00-05:00   46.70   46.70   46.68   46.68  24300.0
2015-01-02 08:22:00-05:00   46.67   46.67   46.67   46.67    200.0
...                           ...     ...     ...     ...      ...
2020-07-28 19:15:00-04:00  202.44  202.44  202.43  202.43    560.0
2020-07-28 19:17:00-04:00  202.43  202.43  202.43  202.43    105.0
2020-07-28 19:18:00-04:00  202.43  202.43  202.43  202.43    247.0
2020-07-28 19:19:00-04:00  202.38  202.38  202.38  202.38    227.0
2020-07-28 19:20:00-04:00  202.35  202.35  202.35  202.35    173.0

[1478998 rows x 5 columns]


In [15]:
five_min_closes = df['close'].resample("5T").mean()

In [18]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [21]:
type(five_min_closes)

pandas.core.series.Series

In [22]:
plot = figure(plot_width=600, plot_height=600, x_axis_type='datetime')

plot.line(x=five_min_closes.index, y=five_min_closes.values)

show(plot)

In [3]:
# Compute moving averages for all combinations of fast and slow windows
fast_ma, slow_ma = vbt.MA.run_combs(
    df['close'], window=np.arange(2, 20), r=2, 
    short_names=['fast', 'slow']
)

In [4]:
# Generate crossover signals for each combination
entries = fast_ma.ma_above(slow_ma, crossed=True)
exits = fast_ma.ma_below(slow_ma, crossed=True)

C:\Users\zachl\anaconda3\envs\alpaca\lib\site-packages\vectorbt\base\accessors.py:411: RuntimeWarning: invalid value encountered in greater
  result = combine_func(new_obj_arr, new_other_arr, *args, **kwargs)
C:\Users\zachl\anaconda3\envs\alpaca\lib\site-packages\vectorbt\base\accessors.py:411: RuntimeWarning: invalid value encountered in less
  result = combine_func(new_obj_arr, new_other_arr, *args, **kwargs)


In [5]:
# Model performance
portfolio = vbt.Portfolio.from_signals(df['close']
                                       , entries, exits, fees=0.001, freq='1D')

# Get total return, reshape to symmetric matrix, and plot the whole thing
portfolio.total_return.vbt.heatmap(
    x_level='fast_window', y_level='slow_window', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%'))
)

MemoryError: Unable to allocate 1.69 GiB for an array with shape (1478998, 153) and data type float64